In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/9shares.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/PG_0.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/agendadaily.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/ajar.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/alhijrahnews-articles.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/aliffchannel.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/amanz.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/asklegal.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/autobuzz.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/azhafizah.com.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/beautifulnara.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/blog.limkitsiang.com.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/dewanbahasa-jdbp.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/discoverkl.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/doctoroncall.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/e-penerbitan.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/fintechnews.my.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/gamerbraves.com.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/harakahdaily.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/iium-confession.jsonl

In [2]:
from glob import glob
import json
import random
from tqdm import tqdm

In [3]:
# files = glob('*.jsonl')
# ls = []
# for f in files:
#     selected = []
#     with open(f) as fopen:
#         for l in fopen:
#             l = json.loads(l)
#             if len(l.split()) > 50:
#                 selected.append((l, f))
#     if len(selected) > 30000:
#         selected = random.sample(selected, 30000)
    
#     ls.extend(selected)

In [4]:
# with open('ls.jsonl', 'w') as fopen:
#     for ls_ in ls:
#         fopen.write(f'{json.dumps(ls_)}\n')

In [5]:
ls = []
with open('ls.jsonl') as fopen:
    for l in fopen:
        ls.append(json.loads(l))
        
len(ls)

213954

In [6]:
ls[0]

['Attack on Titan Brave Order has opened for Pre Registration\n\n\nThe new mobile game Attack on Titan Brave Order from developer Enish has opened for pre-registration\n\n\n\nHere’s the trailer:\n\n\n\nRelated Posts   Pokemon Sleep Now Available In Malaysia   GamerBraves Newsletter Vol. 103 – The Glory of The Barbie/Oppenheimer Double Feature   According to 4Gamer, the first promotional movie has been released, and the pre-registration campaign has started, in which the number of ‘red brilliant’, the currency used for gacha will increase according to the number of registered users.\nHere are the rates:\nFollow us on Twitter\nFollow @GamerBraves\nand Tweet us\nTweet to @GamerBraves\n\n\nOver 30,000 registrations: 2 gacha worth of red brilliant\nOver 50,000 users: 4 gacha worth of red brilliant\nMore than 100,000 players will receive 6 gacha worth of red stones.\nMore than 200,000 users: 8 times the number of gachas\nOver 300,000 users: 10 times the number of gachas\nThe number of red gl

In [7]:
from huggingface_hub import InferenceClient
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=1024,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
    truncate=30000,
)

client = InferenceClient(
    "", timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [8]:
!mkdir summary

mkdir: cannot create directory ‘summary’: File exists


In [ ]:
prompt = f'summarize less than 100 words: {ls[0][0].strip()}'
formatted_prompt = format_prompt(prompt, [])
stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
output = stream.generated_text
output

In [17]:
def answer(q, i):
    filename = f'summary/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = f'summarize less than 100 words: {q[0].strip()}'
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [ ]:
max_worker = 240

questions = ls
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

 91%|██████████████████████████████████████████████████████████████████████████████████████████████▊         | 813/892 [6:48:41<1:08:53, 52.32s/it]

In [9]:
summaries = glob('summary/*.json')
texts = []
for f in tqdm(summaries):
    with open(f) as fopen:
        t = json.load(fopen).strip()
        if len(t.split()) > 10:
            texts.append(t)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 195594/195594 [00:42<00:00, 4551.86it/s]


In [10]:
len(texts)

191787

In [11]:
texts[:10]

['The speaker had recurring nightmares before falling ill, including one of having intimate dreams with unknown men and a nightmare about murder. She also had strange dreams about children, animals, and a cheating husband. She ignored these dreams and considered them just part of her sleep. Her former teacher, who used to bother her, still acted inappropriately after her marriage. Once, he told her that he wanted her, making her uncomfortable. Soon after that, she started experiencing severe stomach pains and abnormal menstrual cycles. She went back to her hometown for treatment and recovered after seven days. When she returned to work, her teacher touched her hand again, which triggered her pain. Despite being unwell, she continued working for three days. On the third day, she suffered a severe abdominal pain and fell in the bathroom. Her husband came from KL that same day to take her back to the mainland for further treatment. Doctors diagnosed her with a magical separation spell, ca

In [12]:
with open('mixtral-summary.texts', 'w') as fopen:
    for t in set(texts):
        fopen.write(f'{json.dumps(t)}\n')

In [14]:
!cp mixtral-summary.texts ../translation